In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name = 'tsbacktest')

Found the config file in: C:\Users\honglu\OneDrive - Microsoft\Projects\ForecastBenchmark\TSPerf\energy_load\GEFCom2017_D_Prob_MT_hourly\submissions\fnn\config.json


In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# choose a name for your cluster
compute_name =  "hlutsperfnn"
compute_min_nodes = 0
compute_max_nodes = 16

vm_size = "STANDARD_D3_V2"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

found compute target. just use it. hlutsperfnn


In [3]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.conda_dependencies import CondaDependencies

env = EnvironmentDefinition()

env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas', 'r-base', 'r-data.table', 'r-rjson', 'r-optparse', 'r-doparallel'], python_version='3.6.2')
env.python.conda_dependencies.add_channel('conda-forge')
env.docker.enabled=True

In [5]:
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import *

script_folder = './'

script_params = {
    '--n_hidden_1': 5, 
    '--n_hidden_2': 5,
    '--iter_max': 3,
    '--penalty': 0,
    '--path': ws.get_default_datastore().as_mount(),
    '--cv_path': ws.get_default_datastore().as_mount()
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='aml_estimator.py',
                environment_definition=env)

ps = GridParameterSampling({
        '--n_hidden_1': choice(4, 8), 
        '--n_hidden_2': choice(4, 8),
        '--iter_max': choice(1, 2, 4, 6, 8, 10),
        '--penalty': choice(0, 0.001),
})

htc2 = HyperDriveRunConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          primary_metric_name='average pinball loss', 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                          max_concurrent_runs=16,
                          max_total_runs=50)
htr2 = exp.submit(config=htc2)

The same input parameter(s) are specified in estimator script params and HyperDrive parameter space. HyperDrive parameter space definition will override duplicate entries in estimator. ['--n_hidden_1', '--n_hidden_2', '--iter_max', '--penalty'] is the list of overridden parameter(s).


In [8]:
from azureml.widgets import RunDetails
RunDetails(htr2).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

2019-01-16 17:35:35,756 WARNING Retrying (Retry(total=2, connect=3, read=3, redirect=None, status=None)) after connection broken by 'SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')",),)': /history/v1.0/subscriptions/ff18d7a8-962a-406c-858f-49acd23d6c01/resourceGroups/hluamlsdkrg/providers/Microsoft.MachineLearningServices/workspaces/hluamlwsnew/experiments/tsbacktest/runs/tsbacktest_1547665964004/children?$orderby=StartTimeUtc%20desc&$top=50
